In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import time
import pandas as pd

            

options = Options()
options.add_argument('--headless=new')
options.add_argument('--mute-audio') 
options.add_argument("--lang=ru-RU")

# Цель проекта

Собрать информацию о youtube каналах, так или иначе связанных с ремонтом, электрикой, сантехникой, стройкой, садоводством, и другим домашним хозяйством.


# Парсим поиск 

In [2]:
queries = ['электрика', 'сантехника', 'плиточник', 'restoration',
           'ремонт авто', 'прокачка авто', 'строительство теплого дома', 'кузница',  'сад и огород', 
              'обустройство дома', 'мебель своими руками', 
          'деревянная мебель', 'покраска стен', 
    'утепление дома', 'работа с гипсокартоном', 'штукатурка', 
    'сварочные работы', 'металлоконструкции', 'вентиляция и кондиционирование', 
    'монтаж натяжных потолков',  'ландшафтный дизайн', 
    'озеленение участка', 
    'организация пространства', 'садоводство', 'строим теплицу']

In [3]:
driver = webdriver.Chrome(options=options)
channels_searched = pd.DataFrame()
consent_accepted = False
for query in queries:
    url = f'https://www.youtube.com/results?search_query={query}'
    try:
        driver.get(url)
    except Exception as e:
        print(f"An error occurred: {e}")
    
    if not consent_accepted:
        consent_button_xpath = "//button[@aria-label='Запретить использование файлов cookie и данных в описанных выше целях']"
        consent = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, consent_button_xpath))
        )
        consent.click()
        consent_accepted = True  

    for _ in range(10): 
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1) 
        
    channels = driver.find_elements(By.CSS_SELECTOR, "ytd-channel-name")
    for channel in channels: 
        try:  
            channel = channel.find_element(By.CSS_SELECTOR, "a.yt-simple-endpoint")
            if channel.text:
                link = channel.get_attribute('href')
                channel_name = channel.text
                temp = pd.DataFrame({'name' : channel_name, 'url' : link, 'query' : query}, index=[0])
                channels_searched = pd.concat([channels_searched, temp]).reset_index(drop=True).drop_duplicates()
        except:
            pass
            

driver.quit()

In [4]:
channels_searched = channels_searched.groupby(['name', 'url'])['query'].unique().reset_index()
channels_searched['query'] = channels_searched['query'].apply(lambda x: ', '.join(x))

In [5]:
channels_searched

,name,url,query
0,#DUDINPH,https://www.youtube.com/@DUDINPh,ремонт авто
1,#DreamTeam Angarsk,https://www.youtube.com/@Pugachev138,сварочные работы
2,#ФОМИН ДИЗАЙН,https://www.youtube.com/@fomin_design,"ландшафтный дизайн, озеленение участка"
3,25 часов в сутках,https://www.youtube.com/@25chasov,"обустройство дома, организация пространства"
4,5-Minute Projects and Design Ideas,https://www.youtube.com/@5minuteprojectideas,сварочные работы
...,...,...,...
678,просто sasha,https://www.youtube.com/@prostosashacrochet,организация пространства
679,строительная компания смит,https://www.youtube.com/@%D1%81%D1%82%D1%80%D0...,сварочные работы
680,“Kapital Realty” Недвижимость Самарканда,https://www.youtube.com/@KapitalRealty,обустройство дома
681,🌺ЭКОПЛАНТ Питомник Растений,https://www.youtube.com/@pitomnikecoplant,"ландшафтный дизайн, озеленение участка, садово..."


# Парсим информацию с каналов

In [6]:
youtube_urls = list(channels_searched.url.drop_duplicates())

In [7]:
driver = webdriver.Chrome(options=options)
all_channels = pd.DataFrame()
consent_accepted = False
length = len(youtube_urls)
i = 0
for url in youtube_urls:
    if i % 100 == 0: 
        print(f'{i} / {length}')
    i+= 1
    name = ''
    is_verified = None
    description = ''
    links = ''
    additional_info_items = ''
    
    try:
        driver.get(url)

        if not consent_accepted:
            consent_button_xpath = "//button[@aria-label='Отклонить все']"
            consent = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, consent_button_xpath))
            )
            consent.click()
            consent_accepted = True  
        
        is_verified = ', подтвержден' in WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.dynamic-text-view-model-wiz__h1'))
        ).get_attribute('aria-label')
        name = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.dynamic-text-view-model-wiz__h1'))
        ).text
        
    
        
        button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.truncated-text-wiz__absolute-button'))
        )
        button.click()
    except Exception as e:
        print(f"An error occurred: {e}")
    

    
    try:
        description = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'description-container'))
        ).find_element(By.CLASS_NAME, 'yt-core-attributed-string').text
    except TimeoutException:
        pass
        
    try:
        links = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'link-list-container'))
        ).find_elements(By.CLASS_NAME, "yt-core-attributed-string__link")
        links = [link.get_attribute('innerHTML') for link in links]
    except TimeoutException:
        pass

    try:
        additional_info_items = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'additional-info-container'))
        ).find_elements(By.CLASS_NAME, "description-item")
        additional_info_items = [item.text for item in additional_info_items if 'посмотреть адрес электронной' not in item.text and item.text != '']
    except TimeoutException:
        pass


    
    
    
    temp_df = pd.DataFrame({'url' : url, 'name' : name, 'is_verified' : is_verified, 'description' : description, 
                            'links' : ', '.join(links), 'info_items': ', '.join(additional_info_items)}, index=[0])
    all_channels = pd.concat([all_channels, temp_df]).reset_index(drop=True)


    

    
driver.quit()

0 / 682
100 / 682
An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF65D1D6CF5+28821]
	(No symbol) [0x00007FF65D143880]
	(No symbol) [0x00007FF65CFE578A]
	(No symbol) [0x00007FF65D0391BE]
	(No symbol) [0x00007FF65D0394AC]
	(No symbol) [0x00007FF65D082647]
	(No symbol) [0x00007FF65D05F33F]
	(No symbol) [0x00007FF65D07F412]
	(No symbol) [0x00007FF65D05F0A3]
	(No symbol) [0x00007FF65D02A778]
	(No symbol) [0x00007FF65D02B8E1]
	GetHandleVerifier [0x00007FF65D50FCED+3408013]
	GetHandleVerifier [0x00007FF65D52745F+3504127]
	GetHandleVerifier [0x00007FF65D51B63D+3455453]
	GetHandleVerifier [0x00007FF65D29BDFB+835995]
	(No symbol) [0x00007FF65D14EB9F]
	(No symbol) [0x00007FF65D14A854]
	(No symbol) [0x00007FF65D14A9ED]
	(No symbol) [0x00007FF65D13A1D9]
	BaseThreadInitThunk [0x00007FFA21577374+20]
	RtlUserThreadStart [0x00007FFA21E3CC91+33]

200 / 682
300 / 682
400 / 682
An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF65D1D6CF5+28821]
	(No symbol) [0x00007

In [8]:
all_channels

,url,name,is_verified,description,links,info_items
0,https://www.youtube.com/@DUDINPh,#DUDINPH,False,,"vk.com/dudinilya, instagram.com/dudinph","www.youtube.com/@DUDINPh, 19 тыс. подписчиков,..."
1,https://www.youtube.com/@Pugachev138,#DreamTeam Angarsk,False,,,"www.youtube.com/@Pugachev138, 379 подписчиков,..."
2,https://www.youtube.com/@fomin_design,#ФОМИН ДИЗАЙН,False,https://www.instagram.com/reel/CosxvdNIbtU/?ig...,,"www.youtube.com/@fomin_design, 51,8 тыс. подпи..."
3,https://www.youtube.com/@25chasov,25 часов в сутках,True,"Всем привет!\n\nCпасибо, что заглянули на мой ...",youtube.com/channel/UCsjkqRa2Bh2iIzabsIh8mJg?s...,"www.youtube.com/@25chasov, 636 тыс. подписчико..."
4,https://www.youtube.com/@5minuteprojectideas,5-Minute Projects and Design Ideas,False,"5-minute projects and design ideas, Furniture ...",bit.ly/2Ea1aqq,"www.youtube.com/@5minuteprojectideas, 270 тыс...."
...,...,...,...,...,...,...
677,https://www.youtube.com/@prostosashacrochet,просто sasha,False,"💌вязание,лайфстаил,распаковки,путешествия💌",,"www.youtube.com/@prostosashacrochet, 116 тыс. ..."
678,https://www.youtube.com/@%D1%81%D1%82%D1%80%D0...,строительная компания смит,False,"Строим дома под ключ,делаем отделку квартир,св...",,"www.youtube.com/@строительнаякомпаниясмит, 8 п..."
679,https://www.youtube.com/@KapitalRealty,“Kapital Realty” Недвижимость Самарканда,False,"Покупай, Продавай или Арендуй в г. Самарканд! ...","t.me/Kapitalrea1ty, instagram.com/invites/cont...","www.youtube.com/@KapitalRealty, 26,5 тыс. подп..."
680,https://www.youtube.com/@pitomnikecoplant,🌺ЭКОПЛАНТ Питомник Растений,False,🔥Неделя сумасшедших скидок до -90% на ВЕСЬ АСС...,"clc.to/Eco-MSK, clc.to/Eco-SPB, clc.to/Eco-Kaz...","www.youtube.com/@pitomnikecoplant, 458 тыс. по..."


In [9]:
all_channels = all_channels.query('name != ""')

In [10]:
all_channels.loc[:,'youtube_link']  = all_channels.info_items.apply(lambda x: x.split(', ')[0].strip())
all_channels.loc[:,'subscribers'] = all_channels.info_items.apply(lambda x: x.split(', ')[1].replace(' подписчиков', '').replace(' подписчика', '').replace(' подписчик', '').strip() if 'подписчик' in x else None)
all_channels.loc[:,'videos_cnt'] = all_channels.info_items.apply(lambda x: x.split(', ')[2].replace(' видео', '').strip() if 'видео' in x else None)
all_channels.loc[:,'views_cnt'] = all_channels.info_items.apply(lambda x: x.split(', ')[3].replace(' просмотров', '').replace(' просмотра', '').replace(' просмотр', '').strip() if 'просмотр' in x else None)
all_channels.loc[:,'reg_date'] = all_channels.info_items.apply(lambda x: x.split(', ')[4].replace('Дата регистрации: ', '').strip() if 'регистрац' in x else None)
all_channels.loc[:,'country'] = all_channels['info_items'].apply(lambda x: x.split(', ')[5] if len(x.split(', ')) > 5 else None)

C:\Users\Asus\AppData\Local\Temp\ipykernel_37476\4033422700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_channels.loc[:,'youtube_link']  = all_channels.info_items.apply(lambda x: x.split(', ')[0].strip())
C:\Users\Asus\AppData\Local\Temp\ipykernel_37476\4033422700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_channels.loc[:,'subscribers'] = all_channels.info_items.apply(lambda x: x.split(', ')[1].replace(' подписчиков', '').replace(' подписчика', '').replace(' подписчик', '').strip() if

In [11]:
all_channels = all_channels.merge(channels_searched[['url', 'query']], on='url')

In [12]:
all_channels

,url,name,is_verified,description,links,info_items,youtube_link,subscribers,videos_cnt,views_cnt,reg_date,country,query
0,https://www.youtube.com/@DUDINPh,#DUDINPH,False,,"vk.com/dudinilya, instagram.com/dudinph","www.youtube.com/@DUDINPh, 19 тыс. подписчиков,...",www.youtube.com/@DUDINPh,19 тыс.,191,23 693 918,4 февр. 2012 г.,Россия,ремонт авто
1,https://www.youtube.com/@Pugachev138,#DreamTeam Angarsk,False,,,"www.youtube.com/@Pugachev138, 379 подписчиков,...",www.youtube.com/@Pugachev138,379,53,62 694,7 янв. 2010 г.,None,сварочные работы
2,https://www.youtube.com/@fomin_design,#ФОМИН ДИЗАЙН,False,https://www.instagram.com/reel/CosxvdNIbtU/?ig...,,"www.youtube.com/@fomin_design, 51,8 тыс. подпи...",www.youtube.com/@fomin_design,"51,8 тыс.",185,25 415 365,21 апр. 2020 г.,None,"ландшафтный дизайн, озеленение участка"
3,https://www.youtube.com/@25chasov,25 часов в сутках,True,"Всем привет!\n\nCпасибо, что заглянули на мой ...",youtube.com/channel/UCsjkqRa2Bh2iIzabsIh8mJg?s...,"www.youtube.com/@25chasov, 636 тыс. подписчико...",www.youtube.com/@25chasov,636 тыс.,262,79 600 356,13 мая 2017 г.,None,"обустройство дома, организация пространства"
4,https://www.youtube.com/@5minuteprojectideas,5-Minute Projects and Design Ideas,False,"5-minute projects and design ideas, Furniture ...",bit.ly/2Ea1aqq,"www.youtube.com/@5minuteprojectideas, 270 тыс....",www.youtube.com/@5minuteprojectideas,270 тыс.,902,41 294 767,21 мая 2009 г.,Соединенные Штаты,сварочные работы
...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,https://www.youtube.com/@prostosashacrochet,просто sasha,False,"💌вязание,лайфстаил,распаковки,путешествия💌",,"www.youtube.com/@prostosashacrochet, 116 тыс. ...",www.youtube.com/@prostosashacrochet,116 тыс.,347,13 085 204,14 сент. 2017 г.,Украина,организация пространства
677,https://www.youtube.com/@%D1%81%D1%82%D1%80%D0...,строительная компания смит,False,"Строим дома под ключ,делаем отделку квартир,св...",,"www.youtube.com/@строительнаякомпаниясмит, 8 п...",www.youtube.com/@строительнаякомпаниясмит,8,19,8 889,3 мая 2014 г.,None,сварочные работы
678,https://www.youtube.com/@KapitalRealty,“Kapital Realty” Недвижимость Самарканда,False,"Покупай, Продавай или Арендуй в г. Самарканд! ...","t.me/Kapitalrea1ty, instagram.com/invites/cont...","www.youtube.com/@KapitalRealty, 26,5 тыс. подп...",www.youtube.com/@KapitalRealty,"26,5 тыс.",976,7 100 926,27 мар. 2017 г.,None,обустройство дома
679,https://www.youtube.com/@pitomnikecoplant,🌺ЭКОПЛАНТ Питомник Растений,False,🔥Неделя сумасшедших скидок до -90% на ВЕСЬ АСС...,"clc.to/Eco-MSK, clc.to/Eco-SPB, clc.to/Eco-Kaz...","www.youtube.com/@pitomnikecoplant, 458 тыс. по...",www.youtube.com/@pitomnikecoplant,458 тыс.,968,148 346 319,24 авг. 2015 г.,Россия,"ландшафтный дизайн, озеленение участка, садово..."


In [13]:
all_channels.to_csv('youtube_channels.csv', index=False)